In [ ]:
!nvidia-smi

Mon Dec  9 13:30:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Init

In [27]:
!pip install transformers

!pip install datasets

!pip install sacrebleu

!pip install torch>=1.13.0

!pip install fsspec==2024.9.0

!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.9 MB/s eta 0:00:00


In [28]:
import random
random.seed(120)

In [29]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [30]:
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


## Load Datasets

In [31]:
BASE_PATH = "/content/gdrive/MyDrive/experiments"
TRAIN_UNRESTRICTED_PATH = f"{BASE_PATH}/unrestricted_train_dataset-50.csv"
TRAIN_RESTRICTED_PATH = f"{BASE_PATH}/restricted_train_dataset.csv"
OOV_UNRESTRICTED_PATH = f"{BASE_PATH}/unrestricted_test_dataset.csv"
OOV_RESTRICTED_PATH = f"{BASE_PATH}/restricted_test_dataset.csv"

In [32]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets


unrestricted_train = load_dataset('csv', data_files=TRAIN_UNRESTRICTED_PATH)
restricted_train = load_dataset('csv', data_files=TRAIN_RESTRICTED_PATH)
unrestricted_test = load_dataset('csv', data_files=OOV_UNRESTRICTED_PATH)
restricted_test = load_dataset('csv', data_files=OOV_RESTRICTED_PATH)

unrestricted_train['train'] = unrestricted_train['train'].add_column('label', [0] * len(unrestricted_train['train']))
unrestricted_test['train'] = unrestricted_test['train'].add_column('label', [0] * len(unrestricted_test['train']))
restricted_train['train'] = restricted_train['train'].add_column('label', [1] * len(restricted_train['train']))
restricted_test['train'] = restricted_test['train'].add_column('label', [1] * len(restricted_test['train']))

train_dataset = concatenate_datasets([unrestricted_train['train'], restricted_train['train']]).shuffle()


test_dataset = concatenate_datasets([unrestricted_test['train'], restricted_test['train']]).shuffle()

dataset = DatasetDict({'train': train_dataset,'test': test_dataset})

dataset

DatasetDict({
    train: Dataset({
        features: ['pair_type', 'ltl', 'en', 'label'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['pair_type', 'ltl', 'en', 'label'],
        num_rows: 15000
    })
})

## Preprocessing

In [33]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [34]:
def preprocess_function(examples):
    return tokenizer(examples["en"], truncation=True)

In [35]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

In [36]:
item = tokenized_dataset['train']['en'][0]
decoded_item = tokenizer.decode(tokenized_dataset['train']['input_ids'][0])

reversed_vocab = {i: w for w, i in tokenizer.get_vocab().items()}

print(item)
print(decoded_item)
print(len(item.split(' ')), len(tokenized_dataset['train']['input_ids'][0]), len(decoded_item.split(' ')))
print(tokenized_dataset['train']['input_ids'][0])
print([reversed_vocab[i] for i in tokenized_dataset['train']['input_ids'][0]])

finally, wltfqPDWHVamwI or zUQhPfeaBWHzFD and, at a certain moment both mOOVh and cMSDnWsocuyuIgu afterwards
[CLS] finally, wltfqpdwhvamwi or zuqhpfeabwhzfd and, at a certain moment both moovh and cmsdnwsocuyuigu afterwards [SEP]
14 45 16
[101, 2633, 1010, 1059, 7096, 2546, 4160, 17299, 2860, 2232, 3567, 2213, 9148, 2030, 16950, 4160, 22269, 7959, 7875, 2860, 2232, 2480, 2546, 2094, 1998, 1010, 2012, 1037, 3056, 2617, 2119, 9587, 4492, 2232, 1998, 4642, 16150, 2078, 9333, 10085, 26230, 10179, 12193, 5728, 102]
['[CLS]', 'finally', ',', 'w', '##lt', '##f', '##q', '##pd', '##w', '##h', '##va', '##m', '##wi', 'or', 'zu', '##q', '##hp', '##fe', '##ab', '##w', '##h', '##z', '##f', '##d', 'and', ',', 'at', 'a', 'certain', 'moment', 'both', 'mo', '##ov', '##h', 'and', 'cm', '##sd', '##n', '##ws', '##oc', '##uy', '##ui', '##gu', 'afterwards', '[SEP]']


In [37]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['pair_type', 'ltl', 'en', 'label', 'input_ids', 'attention_mask'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['pair_type', 'ltl', 'en', 'label', 'input_ids', 'attention_mask'],
        num_rows: 15000
    })
})

In [38]:
tokenized_dataset['train']['label'][49000:49010]

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

## Training

In [59]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [60]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.resize_token_embeddings(len(tokenizer))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(30522, 768, padding_idx=0)

In [61]:
print("For T5:")
print("Tokenizer vocab_size: {}".format(tokenizer.vocab_size))
print("Model vocab size: {}\n".format(model.config.vocab_size))

For T5:
Tokenizer vocab_size: 30522
Model vocab size: 30522



In [62]:
RESULTS_DIR = "/content/gdrive/MyDrive/experiments/classifier"

STEPS = 31

training_args = TrainingArguments(
    output_dir=RESULTS_DIR,
    evaluation_strategy="steps",
    eval_steps=STEPS,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=1,
    # num_train_epochs=1,
    max_steps=STEPS,
    adam_beta2=0.98,
    warmup_steps=100,
    optim="adamw_torch",
    fp16=True,
    save_strategy='steps',
    save_steps=STEPS-1
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [63]:
import numpy as np
import evaluate
from sklearn.metrics import precision_recall_fscore_support

metric = evaluate.load("accuracy")

def compute_metrics(eval_preds):
    labels = eval_preds.label_ids
    preds = eval_preds.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = metric.compute(references=labels, predictions=preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'preds': preds.tolist(),
        'labels': labels.tolist()
    }

In [75]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

#  trainer.train("/content/gdrive/MyDrive/final_thesis_experiments/checkpoint-5625/")
trainer.train()

<ipython-input-75-6ca5b950cea3>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
max_steps is given, it will override any value given in num_train_epochs


Trainer is attempting to log a value of "{'accuracy': 0.9999333333333333}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1

TrainOutput(global_step=31, training_loss=0.507539318453881, metrics={'train_runtime': 23.7817, 'train_samples_per_second': 20.856, 'train_steps_per_second': 1.304, 'total_flos': 5062738392384.0, 'train_loss': 0.507539318453881, 'epoch': 0.00496})

## Test

In [66]:
trainer.evaluate(tokenized_dataset["test"])

Trainer is attempting to log a value of "{'accuracy': 0.9976666666666667}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1

{'eval_loss': 0.5781810283660889,
 'eval_accuracy': {'accuracy': 0.9976666666666667},
 'eval_f1': 0.9976612094888072,
 'eval_precision': 1.0,
 'eval_recall': 0.9953333333333333,
 'eval_preds': [1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1

In [67]:
from transformers import DistilBertForSequenceClassification

In [68]:
list(tokenized_dataset['test']['en'][:300])

['after either VwrjD or GIqieB, it should not happen that GReewNnBI and McJbxXmtny',
 'as a model underfits and the cpu cools down, eventually, the model is underfitted and the plane takes off',
 'sooner or later jemGCguYc or hxHVKZUtB or WojwzxTlL',
 'as either bYqfVJHpOEMXDru or YKbKLZaXZNHo, finally, tFPKy',
 'it is always true that r_SVLBNTCwMwqdk',
 'always, either gpgsEXXnFQJUf, XvnHGoWkxbFo_w or K_byLIlClDIL',
 'when the fridge is switched on then whenever the plane lands then it is going to happen that a mouse captures input afterwards',
 'every time a plane takes off or a cpu overheats or the plane lands',
 'every time either uAIItVRnkSNTlG or CPtYJtPCxB',
 'at no time BwlYH and together FxDtDnJFOv and moUuEAbY',
 'eternally, either a computer starts, the cpu is cooled down or the fridge is switched on',
 'always, either qFZ_JhtW or tiyjLmxH',
 'at any time either dlGWeqWIhWgHAJm or KmQJnzkTH',
 'it should not happen that either gDxAFkvsdjBjyi_ or vWzgP',
 'every time pKEpNPPa

In [74]:
from transformers import T5ForConditionalGeneration

prediction_model = T5ForConditionalGeneration.from_pretrained("/content/gdrive/MyDrive/experiments/classifier/checkpoint-31/")

# model.to('cpu')

src_text = tokenized_dataset['test']['en'][2500:2600] # "translate English to LTL: " + tokenized_dataset['oov']['en'][0]
tokens = tokenizer(src_text, return_tensors="pt", padding=True)
# print(tokenizer([tokenized_dataset['oov']['en'][0]], return_tensors="pt", padding=True))
print(src_text)
print(tokens)

# tok_src = {k: v.to('cuda') for k, v in tokenizer(tokenized_dataset['oov']['en'], return_tensors="pt", padding=True).items()}

# translated = [prediction_model.generate(**tokenizer([t], return_tensors="pt", padding=True), max_length=500) for t in tokenized_dataset['oov']['en']]
translated = prediction_model.generate(**tokens, max_length=256)

# [tokenizer.decode(t, skip_special_tokens=True) for t in translated.tolist()]
# translated.tolist()

You are using a model of type distilbert to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at /content/gdrive/MyDrive/experiments/classifier/checkpoint-31/ and are newly initialized: ['decoder.block.0.layer.0.SelfAttention.k.weight', 'decoder.block.0.layer.0.SelfAttention.o.weight', 'decoder.block.0.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.0.layer.0.SelfAttention.v.weight', 'decoder.block.0.layer.0.layer_norm.weight', 'decoder.block.0.layer.1.EncDecAttention.k.weight', 'decoder.block.0.layer.1.EncDecAttention.o.weight', 'decoder.block.0.layer.1.EncDecAttention.q.weight', 'decoder.block.0.layer.1.EncDecAttention.v.weight', 'decoder.block.0.layer.1.layer_norm.weight', 'decoder.block.0.layer.2.DenseReluDense.wi.weight', 'decoder.block.0.layer.2.DenseReluDense.wo

['it is always true that either a plane takes off, a plane lands or a cpu is cooled down', 'in any case DTCrVFvPNXRfti or EVsSbHwXnNqNMTv', 'after both a computer stops and a cpu is overheated, it will happen that either a model is underfitted or the fridge is empty', 'after both a cpu cools down and the mouse captures input, finally, both the model underfits and the cpu overheats', 'it is always true that a computer starts and, at the same time, the monitor lights up and the mouse clicks', 'absolutely never, the computer starts or the model is underfitted or a model is overfitted', 'all the time a model is underfitted, a plane takes off and the monitor lights up', 'eventually, either fo_pCA or _YBJPgKvYK after nVGNb and eTXcwoikNA', 'after the mouse moves and a fridge is empty, it will happen that the monitor lights up or a plane takes off', 'eventually, both a model underfits and a cpu overheats after a fridge is open and a cpu overheats', 'sooner or later either ySAzm, UrIoDJk or mb

ValueError: `decoder_start_token_id` or `bos_token_id` has to be defined for encoder-decoder generation.

In [ ]:
print(type(translated))

outs_srcs = [tokenizer.decode(t, skip_special_tokens=True) for t in tokenized_dataset['test']['input_ids']]
# outs_lbls = [tokenizer.decode(t, skip_special_tokens=True) for t in translated.predictions]
outs_lbls = [tokenizer.decode(t, skip_special_tokens=True) for t in translated.tolist()]

outs_srcs[:2], outs_lbls[:2]

In [ ]:
print(tokenized_dataset['test']['ltl'][2500:2510])
print(outs_lbls[:10])

print(type(tokenized_dataset['test']['ltl']))
print(type(outs_lbls))

metric.compute(references=[[t] for t in tokenized_dataset['test']['ltl'][:100]], predictions=outs_lbls)